In [1]:
import pandas as pd

In [2]:

#importing raw data
from google.colab import files
uploaded = files.upload()


Saving ratings_small.csv to ratings_small.csv


In [3]:
import io
Raw_data = pd.read_csv(io.BytesIO(uploaded['ratings_small.csv']))


In [4]:
Raw_data.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [5]:
#cleaning the data
#check for null values in the movie id and ratings column
Raw_data[['movieId', 'rating','userId']].isnull().sum()

movieId    0
rating     0
userId     0
dtype: int64

In [6]:
#create a dataframe with ratings as columns and users for rows
import numpy as np
from scipy.sparse import csr_matrix
Data=Raw_data
SVD_matrix_pivot=pd.pivot(data=Data, index='movieId', columns='userId', values= 'rating').fillna(0)



In [7]:
#exploring the newly created data frame
SVD_matrix_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,...,0.0,4.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#create a matrix using csr matrix
import scipy
csr_matrix=scipy.sparse.csr_matrix(SVD_matrix_pivot.values)

In [9]:
#Normalize the matrix
from sklearn.preprocessing import normalize
#normalize rows of the matrix
normalize(csr_matrix, axis=1, norm='l1')
#normalize columns of the matrix
normalize(csr_matrix, axis=0, norm='l1')

<9066x671 sparse matrix of type '<class 'numpy.float64'>'
	with 100004 stored elements in Compressed Sparse Column format>

In [10]:
#computing singular value decomposition
import numpy as np                                                                                   
from scipy import linalg                                                                            
from scipy.sparse import linalg as slinalg 

SVD_Matrix=scipy.sparse.linalg.svds(csr_matrix)


In [11]:
#getting the U, S and Vt values
U, sigma, Vt = scipy.sparse.linalg.svds(csr_matrix, k = 10)

In [12]:
#Sigma value above is outputed as an array, but we need it in the form of a diagonal matrix
sigma = np.diag(sigma)

In [13]:
#creating predictions
user_means = np.array(csr_matrix.mean(axis = 1)).reshape(-1, 1)
predicted = np.dot(np.dot(U, sigma), Vt)
predicted_ratings = np.dot(np.dot(U, sigma), Vt) * user_means
predicted_ratings

array([[-1.02035555e-02,  2.30678667e+00,  1.58394723e+00, ...,
         3.07958728e-01,  1.14021636e+00,  3.26433950e+00],
       [-4.06094674e-04,  8.57740656e-01,  2.22922795e-01, ...,
         2.13006502e-02,  1.82682448e-01,  2.04933275e-01],
       [ 5.27595588e-03,  1.75586825e-01,  2.41641347e-02, ...,
         8.42754799e-03,  3.14793205e-02,  7.67120993e-03],
       ...,
       [-5.96296219e-06, -2.85455364e-06,  2.75718356e-05, ...,
        -2.31369308e-07,  1.00887832e-05,  4.38717146e-05],
       [-2.14666639e-06, -1.02763931e-06,  9.92586083e-06, ...,
        -8.32929510e-08,  3.63196194e-06,  1.57938173e-05],
       [ 1.45155279e-05,  1.56798734e-04, -2.08679679e-04, ...,
        -1.41843658e-04, -1.87356383e-04, -6.44325990e-04]])

In [17]:
#create new data frame and include predicted ratings
#code to include column titles
#df_predict = pd.DataFrame(predicted_ratings, columns= SVD_matrix_pivot.columns, index=SVD_matrix_pivot.index)
#code excluding column titles
df_predict = pd.DataFrame(predicted_ratings, index=SVD_matrix_pivot.index)
#delete index title
df_predict = df_predict.rename_axis(None, axis=0)
# If Data frame index starts with 0 but original dataset starts with 1 add 1 to index
# code: predicted_df.index = predicted_df.index + 1
df_predict.head()

,0,1,2,3,4,5,6,7,8,9,...,661,662,663,664,665,666,667,668,669,670
1,-0.010204,2.306787,1.583947,3.412151,2.181016,0.493282,2.547858,2.898374,1.083075,0.717927,...,2.245010,0.623011,4.753910,4.920413,1.412024,1.569910,0.494373,0.307959,1.140216,3.264340
2,-0.000406,0.857741,0.222923,0.629456,0.457497,0.011484,0.538470,0.139453,0.112283,0.030399,...,0.806905,0.102280,0.633107,1.185294,0.486952,0.585998,0.027037,0.021301,0.182682,0.204933
3,0.005276,0.175587,0.024164,0.173402,0.073276,-0.002522,0.109183,-0.007715,0.018987,0.008977,...,0.157597,0.011202,0.037338,0.248312,0.095922,0.126755,0.002935,0.008428,0.031479,0.007671
4,0.000176,0.008232,0.000941,0.005541,0.004793,0.000268,0.004195,-0.001487,0.000761,-0.000759,...,0.007577,0.000106,-0.007398,0.010066,0.004298,0.006222,0.000642,0.000126,0.001543,0.001131
5,0.000019,0.169674,0.031744,0.053676,0.211587,-0.001319,0.020169,0.028366,0.036458,-0.023565,...,0.179461,0.001394,-0.042035,0.484818,0.087010,0.106601,-0.005564,0.010501,0.023801,0.055721


In [18]:
userId = int(input("Enter user number: "))

Enter user number: 12


In [19]:
num_recommended_movies = int(input("Enter number of movies to be recommended: "))

Enter number of movies to be recommended: 5


In [59]:
#Function to recommend movies
def recommend_movies(userId, num_recommended_movies):
  #watched movies
  df=SVD_matrix_pivot
  df.drop(df.index[df[userId] == 0.0], inplace = True)
  #Drop the watched movies to remain with watched movies only
  df_predict.drop(df.index)
  #Get the user number ratings of unwatched movies
  unwatched_movies_ratings=df_predict.loc[:,userId]
  #sort out the ratings in descending order
  unwatched_movies_ratings_sorted=unwatched_movies_ratings.sort_values(ascending=False)
  #List number of recommended movies
  movie_ratings=unwatched_movies_ratings_sorted[ :num_recommended_movies]
  #Get the index of recommended movies
  Recommended_movies=movie_ratings.index
  return Recommended_movies

In [56]:
recommend_movies(userId, num_recommended_movies)

Int64Index([318, 356, 296, 593, 2571], dtype='int64')

In [57]:
Recommended_Movies=recommend_movies(userId, num_recommended_movies).tolist()

In [58]:
Recommended_Movies

[318, 356, 296, 593, 2571]